<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup
from config import DB_CONFIG

✅ Конфиг загружен: {'dbname': '***', 'user': '***', 'password': '***', 'host': '***', 'port': '***'}


In [6]:
def create_connection():
    """Безопасное подключение к PostgreSQL"""
    try:
        conn = psycopg2.connect(
            dbname=DB_CONFIG["dbname"],
            user=DB_CONFIG["user"],
            password=DB_CONFIG["password"],
            host=DB_CONFIG["host"],
            port=DB_CONFIG["port"]
        )
        print("✅ Подключение успешно")
        return conn
    except Exception as e:
        print(f"❌ Ошибка подключения: {e}")
        return None

connection = create_connection()

✅ Подключение успешно


In [7]:
print("Параметры подключения:",  DB_CONFIG.items())

Параметры подключения: dict_items([('dbname', 'project_sql'), ('user', 'skillfactory'), ('password', 'cCkxxLVrDE8EbvjueeMedPKt'), ('host', '84.201.134.129'), ('port', 5432)])


# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [8]:
# SQL запрос для получения количества вакансий
query = """
         SELECT count(id)
         FROM public.VACANCIES
"""

In [9]:
# Выполнение запроса и загрузка результата в DataFrame
df = pd.read_sql_query(query, connection)

# Извлечение числа из результата и вывод его
number_of_vacancies = df.iloc[0, 0]
print(f"Вакансий в базе данных: {number_of_vacancies}")

Вакансий в базе данных: 49197


/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_7885/1532738938.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [ ]:
query = """
         SELECT count(id)
         FROM public.EMPLOYERS
"""

In [ ]:
# Выполнение запроса и загрузка результата в DataFrame
df = pd.read_sql_query(query, connection)

# Извлечение числа из результата и вывод его
count_of_employers= df.iloc[0, 0]
print(f"работодателей в базе данных: {number_of_vacancies}")

работодателей в базе данных: 49197


/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_2718/3534574006.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [ ]:
query = """
         SELECT count(id)
         FROM public.AREAS
"""

In [ ]:
# Выполнение запроса и загрузка результата в DataFrame
df = pd.read_sql_query(query, connection)

# Извлечение числа из результата и вывод его
regions = df.iloc[0, 0]
print(f"регионов в базе данных: {regions}")

регионов в базе данных: 1362


/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_2718/2764238379.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [ ]:
query = """
         SELECT count(id)
         FROM public.INDUSTRIES
"""

In [ ]:
# Выполнение запроса и загрузка результата в DataFrame
df = pd.read_sql_query(query, connection)

# Извлечение числа из результата и вывод его
count_of_industry_sectors = df.iloc[0, 0]
print(f"сфер деятельности в базе данных: {count_of_industry_sectors}")

сфер деятельности в базе данных: 294


/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_2718/3883410565.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Для предварительного анализа данных можно сделать несколько основных выводов из полученной информации:

1. **Количество вакансий**: Вы получили общее количество записей о вакансиях в базе данных. Это число показывает общую активность на рынке труда, охваченную вашей базой данных.

2. **Количество работодателей**: Это число является индикатором разнообразия и насыщенности рынка труда, так как показывает, сколько различных работодателей представлено в вашей базе данных.

3. **Количество регионов**: Число регионов в вашей базе данных может свидетельствовать о географическом охвате собранной информации. Чем больше регионов, тем шире охват.

4. **Количество сфер деятельности**: Это число указывает на разнообразие индустрий и отраслей, представленных в вашей базе данных. Оно может помочь понять, какие сектора экономики наиболее активны в плане вакансий.

***

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [ ]:
query = """
        SELECT
                a.name as area,
                COUNT(v.id)
        FROM public.AREAS as a
                join public.VACANCIES as v on a.id = v.area_id
        GROUP BY a.name 
        ORDER BY COUNT(v.id) DESC;
"""

In [ ]:
df = pd.read_sql_query(query, connection)
display(df)

/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_2718/3475273297.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,area,count
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [ ]:
query = """
         SELECT count(id) as cnt
         FROM public.VACANCIES
         WHERE (salary_from is not NULL) or (salary_to is not NULL)
"""

In [ ]:
df = pd.read_sql_query(query, connection)

count_of_vacancies = df.iloc[0, 0]
print(f"количества вакансий у которых заполнено хотя бы одно из двух полей с зарплатой: {number_of_vacancies}")

/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_2718/3627844437.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


количества вакансий у которых заполнено хотя бы одно из двух полей с зарплатой: 49197


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [ ]:
query = """
         SELECT 
            AVG(salary_from) as mean_salary_from,
            AVG(salary_to) as mean_salary_to
         FROM public.VACANCIES
"""

In [ ]:
df = pd.read_sql_query(query, connection)

mean_salary_from = df.iloc[0, 0]
mean_salary_to = df.iloc[0, 1]
print(f"""средние значения для нижней границы зарплатной вилки: {round(mean_salary_from)},
средние значения для верхней границы зарплатной вилки: {round(mean_salary_to)}""")

/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_2718/2645030245.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


средние значения для нижней границы зарплатной вилки: 71065,
средние значения для верхней границы зарплатной вилки: 110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [ ]:
query = """
         SELECT
            schedule,
            employment,
            count(id)
         FROM public.VACANCIES
         GROUP BY 
            schedule,
            employment
        ORDER BY
            count(id) DESC
"""

In [ ]:
df = pd.read_sql_query(query, connection)
display(df)

/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_2718/3475273297.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [ ]:
query = """
         SELECT 
            experience,
            COUNT(id)
         FROM public.VACANCIES
         GROUP BY
            experience
         ORDER BY 
            COUNT(id) ASC
            
"""

In [ ]:
df = pd.read_sql_query(query, connection)
display(df)

/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_2718/3475273297.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


На основе ваших данных можно сделать несколько выводов:

1. **Распределение вакансий по регионам:**
   - Москва и Санкт-Петербург значительно лидируют по количеству вакансий с 5333 и 2851 вакансиями соответственно.
   - Такие крупные города, как Минск и Новосибирск, также имеют значительное число вакансий, с более чем 2000 каждой.
   - В регионах, таких как Тарко-Сале и Новоаннинский, количество вакансий значительно ниже, вплоть до единичных значений, что может свидетельствовать о меньшем количестве рабочих мест или специфике рынка труда в этих районах.

2. **Зарплата:**
   - Из 24073 вакансий, где указана зарплата, средняя нижняя граница составляет 71065, а верхняя — 110537. Это говорит о довольно широком диапазоне предложений и может указывать на разнообразие отраслей и уровней квалификации, доступных для соискателей.

3. **Распределение вакансий по графику и форме занятости:**
   - Полный день с полной занятостью является наиболее распространенным графиком, с 35367 вакансий.
   - Удаленная работа также существенно представлена, особенно в форме полной (7802 вакансии) и частичной занятости (1312 вакансий), что подчеркивает тренд на гибкость в работе.
   - Стажировки и проектная работа возрастают в популярности, свидетельствуя об интересе работодателей к гибким и временным формам найма.

4. **Опыт работы:**
   - Большая часть вакансий рассчитана на кандидатов с опытом от 1 до 3 лет (26152 вакансии).
   - Вакансии для новичков тоже актуальны, с 7197 предложениями, что может способствовать привлечению молодых специалистов и выпускников.
   - Требования к кандидатам с опытом более 6 лет составляют лишь 1337 вакансий, что предполагает меньшую потребность в высококвалифицированных кадрах или же насыщенность рынка такими специалистами.

***

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [ ]:
query = """
        SELECT
            e.name,
            COUNT(v.id)
        FROM public.EMPLOYERS as e
            join public.VACANCIES as v on v.employer_id = e.id
        GROUP BY
            e.name
        ORDER BY
            COUNT(v.id) DESC
        LIMIT 5
"""

In [ ]:
df = pd.read_sql_query(query, connection)
display(df)

/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_2718/3475273297.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [ ]:
query = """
        SELECT
            a.name,
            COUNT(e.id) as EMPLOYERS_COUNT,
            COUNT(v.id) as VACANCIES_COUNT
        FROM (public.AREAS as a
            left join public.EMPLOYERS as e on a.id = e.area)
            left join public.VACANCIES as v on a.id = v.area_id
        GROUP BY
            a.name
        HAVING
            COUNT(v.id) = 0
        ORDER BY
            COUNT(e.id) DESC
        LIMIT 1
"""

In [ ]:
df = pd.read_sql_query(query, connection)

region = df.iloc[0, 0]
print(f"регион в котором нет вакансий, но самое большое количество работодателей: {region}")

/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_2718/575047205.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


регион в котором нет вакансий, но самое большое количество работодателей: Россия


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [ ]:
query = """
        SELECT 
            e.name,
            COUNT(DISTINCT a.id) as region_count
        FROM (public.VACANCIES as v
            left join public.AREAS as a on a.id = v.area_id)
            left join public.EMPLOYERS as e on v.employer_id = e.id
        GROUP BY 
            e.name
        ORDER BY
            COUNT(a.id) DESC
"""

In [ ]:
df = pd.read_sql_query(query, connection)
display(df)

/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_2718/3475273297.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,name,region_count
0,Яндекс,181
1,Ростелеком,152
2,Тинькофф,43
3,СБЕР,24
4,Газпром нефть,24
...,...,...
14761,ПСБ Инновации и Инвестиции,1
14762,ПСЗ Янтарь,1
14763,ПСК,1
14764,Псковский завод механических приводов,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [ ]:
query = """
        SELECT
            COUNT(e.id)
        FROM public.EMPLOYERS as e
            left join public.EMPLOYERS_INDUSTRIES as ei on e.id = ei.employer_id
        WHERE 
            ei.industry_id is null
"""

In [ ]:
value = pd.read_sql_query(query, connection).iloc[0, 0]
display(f"количества работодателей, у которых не указана сфера деятельности: {value}")

/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_2718/1796317306.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  value = pd.read_sql_query(query, connection).iloc[0, 0]


'количества работодателей, у которых не указана сфера деятельности: 8419'

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [ ]:
query = """
        SELECT 
            e.name, 
            COUNT(ei.industry_id)
        FROM public.EMPLOYERS as e
            left join public.EMPLOYERS_INDUSTRIES as ei ON e.id = ei.employer_id
        GROUP BY 
            e.id
        HAVING 
            COUNT(ei.industry_id) = 4
        ORDER BY 
            name
        OFFSET 2
        LIMIT 1
"""

In [ ]:
value = pd.read_sql_query(query, connection).iloc[0, 0]
print(f"""Название компании, находящейся на третьем месте в алфавитном списке: {value}""")

Название компании, находящейся на третьем месте в алфавитном списке: 2ГИС


/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_2718/577246709.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  value = pd.read_sql_query(query, connection).iloc[0, 0]


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [ ]:
query = """
        SELECT 
            COUNT(DISTINCT e.id)
        FROM (public.EMPLOYERS as e
            join public.EMPLOYERS_INDUSTRIES as ei ON e.id = ei.employer_id)
            join public.INDUSTRIES as i ON i.id = ei.industry_id
        WHERE 
            i.name = 'Разработка программного обеспечения'
"""

In [ ]:
df = pd.read_sql_query(query, connection).iloc[0, 0]
print(f"Этот запрос подсчитает количество уникальных работодателей, связанных с индустрией 'Разработка программного обеспечения': {df}" )

Этот запрос подсчитает количество уникальных работодателей, связанных с индустрией 'Разработка программного обеспечения': 3553


/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_2718/891658512.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection).iloc[0, 0]


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [ ]:
# Получаем список городов-миллионников
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Находим таблицу с городами
tables = soup.find_all('table', {'class': 'standard sortable'})
million_cities = []

# Берем первую таблицу и извлекаем города
table = tables[0]
for row in table.find_all('tr')[1:]:
    city = row.find('a')
    million_cities.append(city.text)

print(million_cities)

['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск', 'Уфа', 'Самара', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград']


In [ ]:
# Запрос для получения информации о вакансиях компании "Яндекс"
query = f"""
WITH RegionVacancyCount AS (
  SELECT
    a.name AS area_name,
    COUNT(v.id) AS vacancy_count
  FROM AREAS a
  JOIN VACANCIES v ON a.id = v.area_id
  JOIN EMPLOYERS e ON v.employer_id = e.id
  WHERE e.name = 'Яндекс'
  GROUP BY a.name
)

SELECT
  area_name,
  vacancy_count
FROM RegionVacancyCount
WHERE area_name IN ({','.join(["'" + city + "'" for city in million_cities])})

UNION ALL

SELECT
  'Total',
  SUM(vacancy_count)
FROM RegionVacancyCount
WHERE area_name IN ({','.join(["'" + city + "'" for city in million_cities])})
"""

# Выполнение запроса и получение данных в виде DataFrame
df = pd.read_sql(query, connection)

# Вывод DataFrame
print(f"""{df.shape[0]} строк получилось в нашей выборке
результат в тотал = {df[df["area_name"] == "Total"].iloc[0, 1]}"""
      )

/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_2718/1078715541.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


17 строк получилось в нашей выборке
результат в тотал = 485.0


***

In [ ]:
# выводы по анализу работодателей

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [ ]:
query = """
        SELECT COUNT(*)
        FROM vacancies
        WHERE 
                LOWER(name) LIKE '%data%' OR
                LOWER(name) LIKE '%данн%'
"""

In [ ]:
vacancy_count = pd.read_sql_query(query, connection).iloc[0, 0]
print(f"Количество вакансий, относящихся к данным: {vacancy_count}")

Количество вакансий, относящихся к данным: 1771


/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_2718/3107297287.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vacancy_count = pd.read_sql_query(query, connection).iloc[0, 0]


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [ ]:
query = """
         SELECT COUNT(*)
        FROM vacancies
        WHERE (
            name ILIKE '%data scientist%' OR
            name ILIKE '%data science%' OR
            name ILIKE '%исследователь данных%' OR
            name ILIKE '%ML%' AND name NOT ILIKE '%HTML%' OR
            name ILIKE '%machine learning%' OR
            name ILIKE '%машинн%обучен%'
            ) AND (
            name ILIKE '%junior%' OR
            experience = 'Нет опыта' OR
            employment = 'Стажировка'
)
"""

In [ ]:
vacancy_count = pd.read_sql_query(query, connection).iloc[0, 0]
print(f"Количество вакансий, подходящих для начинающего дата-сайентиста {vacancy_count}")

/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_2718/1752682198.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vacancy_count = pd.read_sql_query(query, connection).iloc[0, 0]


Количество вакансий, подходящих для начинающего дата-сайентиста 51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
query = """
        SELECT 
            v.id,
            v.key_skills
        FROM vacancies v
        WHERE( 
            (v.key_skills like '%SQL%') OR
            (v.key_skills ilike '%postgres%')
                )
            AND
            (
            (name ILIKE '%data scientist%')
            OR (name ILIKE '%data science%')
            OR (name ILIKE '%исследователь данных%')
            OR ((name ILIKE '%ML%') AND (name NOT ILIKE '%HTML%'))
            OR (name ILIKE '%machine learning%')
            OR (name ILIKE '%машинн%обучен%')
            AND (key_skills IS NOT NULL)
            )
"""

In [ ]:
vacancy_count = pd.read_sql_query(query, connection)
print(f"Количество подходящих вакансий:  {vacancy_count.shape[0]}")

/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_2718/2541778471.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vacancy_count = pd.read_sql_query(query, connection)


Количество подходящих вакансий:  229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
query = """
        SELECT 
                v.id,
                v.key_skills
        FROM vacancies v
        WHERE( 
                (v.key_skills like '%Python%')
                )
                AND
                (
                (name ILIKE '%data scientist%') OR (name ILIKE '%data science%') OR (name ILIKE '%исследователь данных%')
                OR ((name ILIKE '%ML%') AND (name NOT ILIKE '%HTML%'))
                OR (name ILIKE '%machine learning%')
                OR (name ILIKE '%машинн%обучен%')
                AND (key_skills IS NOT NULL)
                )
"""

In [ ]:
vacancy_count = pd.read_sql_query(query, connection)
print(f"Количество подходящих вакансий:  {vacancy_count.shape[0]}")

Количество подходящих вакансий:  357


/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_2718/2541778471.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vacancy_count = pd.read_sql_query(query, connection)


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
query = f'''
    SELECT
        ROUND(AVG(LENGTH(v.key_skills) - LENGTH(REPLACE(v.key_skills, CHR(9), '')) + 1), 2)
    FROM
        vacancies v
    WHERE
    (
        LOWER(v.name) LIKE '%data scientist%'
        OR LOWER(v.name) LIKE '%data science%'
        OR LOWER(v.name) LIKE '%machine learning%'
        OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
        OR LOWER(v.name) LIKE '%исследователь данных%'
        OR LOWER(v.name) LIKE '%машинн%обучен%'
    )
    AND
    (
        v.key_skills IS NOT NULL
    )
'''

In [ ]:
vacancy = pd.read_sql_query(query, connection)
print(f"навыков в среднем указывают в вакансиях для DS:  {vacancy}")

/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_2718/478506846.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vacancy = pd.read_sql_query(query, connection)


навыков в среднем указывают в вакансиях для DS:     round
0   6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [ ]:
query = f"""
    SELECT
  v.experience,
  ROUND(AVG(coalesce((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to)), 0)
FROM vacancies v
WHERE
(
       (v.name ILIKE '%data scientist%')
    OR (lower(v.name) ILIKE '%data science%')
    OR (v.name ILIKE '%исследователь данных%')
    OR (v.name  LIKE '%ML%' AND  (v.name NOT iLIKE '%html%'))
    OR (v.name ILIKE '%machine learning%')
    OR (v.name ILIKE '%машинн%обучен%')
    AND (key_skills IS NOT NULL)
)
GROUP BY v.experience
ORDER BY 1"""
    
"""select

  v.experience, ROUND(AVG(coalesce((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to)), 0)

from vacancies v

WHERE
(
       (v.name ILIKE '%data scientist%')
    OR (lower(v.name) ILIKE '%data science%')
    OR (v.name ILIKE '%исследователь данных%')
    OR (v.name  LIKE '%ML%' AND  (v.name NOT iLIKE '%html%'))
    OR (v.name ILIKE '%machine learning%')
    OR (v.name ILIKE '%машинн%обучен%')
    AND (key_skills IS NOT NULL)
)

GROUP BY v.experience
ORDER BY 1"""

"select\n\n  v.experience, ROUND(AVG(coalesce((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to)), 0)\n\nfrom vacancies v\n\nWHERE\n(\n       (v.name ILIKE '%data scientist%')\n    OR (lower(v.name) ILIKE '%data science%')\n    OR (v.name ILIKE '%исследователь данных%')\n    OR (v.name  LIKE '%ML%' AND  (v.name NOT iLIKE '%html%'))\n    OR (v.name ILIKE '%machine learning%')\n    OR (v.name ILIKE '%машинн%обучен%')\n    AND (key_skills IS NOT NULL)\n)\n\nGROUP BY v.experience\nORDER BY 1"

In [ ]:
df = pd.read_sql_query(query, connection)
print(df)

/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_2718/2479061017.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


           experience     round
0         Более 6 лет       NaN
1           Нет опыта   74643.0
2  От 1 года до 3 лет  145236.0
3       От 3 до 6 лет  243115.0


***

# Общий вывод по проекту

In [12]:
query = f"""
    SELECT DISTINCT name
    FROM industries
"""

In [14]:
df = pd.read_sql_query(query, connection)
display(df.tail(20))

/var/folders/mq/mckdrm5j7zn2z8xvrsj5_s180000gn/T/ipykernel_7885/1352746026.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,name
274,Судоремонт
275,Энергоснабжение
276,"Автокомпоненты, запчасти (производство)"
277,Складские услуги
278,Управление и эксплуатация недвижимости
279,Тренинговые компании
280,Розничная торговля автомобилями (дилерский центр)
281,Интернет-провайдер
282,"Морские, речные перевозки"
283,"Управляющая компания группы, холдинга, штаб-кв..."
